In [23]:
from antlr4 import *
from SQLiteLexer import SQLiteLexer
from SQLiteParser import SQLiteParser

input_file = "examples/sql1.sql"

input = FileStream(input_file)
lexer = SQLiteLexer(input)
stream = CommonTokenStream(lexer)
parser = SQLiteParser(stream)
tree = parser.parse()
print(tree.toStringTree(recog=parser))

(parse (sql_stmt_list (sql_stmt (select_stmt (select_core select (result_column *) from (table_or_subquery (table_name (any_name x))))))) <EOF>)
